In [71]:
import pandas as pd
import features
import glob,re,pickle,os,yaml,datetime

In [2]:
debut=datetime.datetime.now()
filePrefix="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/LongitudinalesRnd/"
sampleFiles=glob.glob(filePrefix+"Longitudinal*.csv")

In [3]:
def prefixEchantillon(numero):
    candidats=[]
    for sample in sampleFiles:
        m=re.match(ur"^.*/(Longitudinal-%s-T\d+-F\d+)%s-paradigmes\.csv"%(numero,casesType),sample)
        if m:
            candidats.append(m.group(1))
    if len(candidats)==1:
        return candidats[0]
    else:
        print "PB pas de nom unique correspondant",candidats

In [4]:
numeroEchantillon="00"
sampleType="-X"
casesType="-X-Morphomes"
etapeSwim="-Swim1"
samplePrefix=prefixEchantillon(numeroEchantillon)
initialFile=filePrefix+samplePrefix+"-X-paradigmes.csv"
analysisPrefix=filePrefix+samplePrefix+casesType
predictionsFile=analysisPrefix+"-paradigmes%s.csv"%etapeSwim
print predictionsFile
referenceFile="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/MGC-171229-Verbes3.pkl"

/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/LongitudinalesRnd/Longitudinal-00-T1000000-F29754-X-Morphomes-paradigmes-Swim1.csv


## Réglages de la variante phonologique

In [5]:
phonologicalMap=sampleType.strip("-")
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [6]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [7]:
dierese={"j":"ij", "w":"uw","H":"yH","i":"ij","u":"uw","y":"yH"}

In [8]:
def checkFrench(prononciation):
    glide2voyelle={"j":"i","w":"u","H":"y"}
    if prononciation:
        if "," in prononciation:
            prononciations=prononciation.split(",")
            setPrononciations=set()
            for element in prononciations:
                setPrononciations.add(checkFrench(element))
            result=",".join(list(setPrononciations))
        else:
            result=recoder(prononciation)
            result=result.replace("Jj","J")
            m=re.match(ur"^(.*[^ieèEaOouy926êôâ])([jwH])$",result)
            if m:
                print ("pb avec un glide final", prononciation)
                voyelle=glide2voyelle[m.group(2)]
                result=m.group(1)+voyelle
            m=re.match(ur"(.*[ptkbdgfsSvzZ][rl])([jwH])(.*)",result)
            if m:
                n=re.search(ur"[ptkbdgfsSvzZ][rl](wa|Hi|wê)",result)
                if not n:
                    glide=m.group(2)
                    result=m.group(1)+dierese[glide]+m.group(3)
            m=re.match(ur"(.*)([iuy])([ieEaOouy].*)",result)
            if m:
                glide=m.group(2)
                result=m.group(1)+dierese[glide]+m.group(3)
            result=result.replace("jj","ij")
    else:
        result=prononciation
    return result

In [9]:
print checkFrench(u"kopjjô")

kopijô


## Formes de l'échantillon

In [10]:
initialParadigmes=pd.read_csv(initialFile,sep=";",encoding="utf8")
del initialParadigmes[u"Unnamed: 0"]
initialParadigmes=initialParadigmes.dropna(axis=1,how='all')
initialParadigmesColumns=initialParadigmes.columns.tolist()
listeLexemes=initialParadigmes["lexeme"].tolist()
nbLexemes=len(listeLexemes)

In [11]:
initialForms=pd.melt(initialParadigmes[initialParadigmes["lexeme"].isin(listeLexemes)],id_vars=["lexeme"]).dropna()
initialForms["lexeme-case"]=initialForms["lexeme"]+"-"+initialForms["variable"]
initialForms.drop(labels=["lexeme","variable"],axis=1,inplace=True)
initialForms.set_index(["lexeme-case"],inplace=True)
initialFormsIndex=initialForms.index.tolist()

## Formes de l'échantillon avec les prédictions

In [12]:
predictedParadigmes=pd.read_csv(predictionsFile,sep=";",encoding="utf8")
del predictedParadigmes[u"Unnamed: 0"]
predictedParadigmes=predictedParadigmes.loc[:,predictedParadigmes.columns.isin(initialParadigmesColumns)].dropna(axis=1,how='all')
if listeLexemes!=predictedParadigmes["lexeme"].tolist():
    print "PB avec la liste des lexèmes prédits"
if set(initialParadigmesColumns)!=set(predictedParadigmes.columns.tolist()):
    print "PB avec la liste des cases prédites"
    print predictedParadigmes.columns.tolist()
    print initialParadigmesColumns

In [13]:
predictedForms=pd.melt(predictedParadigmes[predictedParadigmes["lexeme"].isin(listeLexemes)],id_vars=["lexeme"]).dropna()
predictedForms["lexeme-case"]=predictedForms["lexeme"]+"-"+predictedForms["variable"]
predictedForms.drop(labels=["lexeme","variable"],axis=1,inplace=True)
predictedForms.set_index(["lexeme-case"],inplace=True)
predictedFormsIndex=predictedForms.index.tolist()

In [14]:
predictedForms

,value
lexeme-case,
abandonner-ai1P,abâdOnam
aberrer-ai1P,abEram
ablater-ai1P,ablatam
abloquer-ai1P,ablOkam
abonner-ai1P,abOnam
aborder-ai1P,abOrdam
aborner-ai1P,abOrnam
abouter-ai1P,abutam
abraser-ai1P,abrazam


## Formes de référence

In [15]:
with open(referenceFile,"rb") as input:
    lexiqueGold=pickle.load(input)

In [16]:
lexiqueGold[lexiqueGold["lexeme"]=="copier"]

,ortho,phono,ext,cs,ms,vs,lexeme,freq,prob,case
86040,copia,kOpja,NaN,V,3S,ai,copier,8.000061e+06,2.317306e-07,ai3S
86043,copiai,kOpjE,NaN,V,1S,ai,copier,7.000007e+06,2.027630e-07,ai1S
86044,copiaient,kOpjE,"t""",V,3P,ii,copier,2.800003e+07,8.110518e-07,ii3P
86045,copiais,kOpjE,"z""",V,1S,ii,copier,3.614247e+07,1.046907e-06,ii1S
86046,copiais,kOpjE,"z""",V,2S,ii,copier,8.575646e+05,2.484031e-08,ii2S
86047,copiait,kOpjE,"t""",V,3S,ii,copier,4.800014e+07,1.390377e-06,ii3S
86048,copiant,kOpjâ,"t""",V,,pP,copier,4.400003e+07,1.274510e-06,pP
86049,copias,kOpja,"z""",V,2S,ai,copier,2.039116e-01,5.906524e-15,ai2S
86050,copiasse,kOpjas,@,V,1S,is,copier,1.500807e-01,4.347254e-15,is1S
86051,copiassent,kOpjas,"@t""",V,3P,is,copier,2.523647e-01,7.310024e-15,is3P


In [17]:
'''Rectifications phonologiques'''
lexiqueGold["phono"]=lexiqueGold["phono"].apply(lambda x: checkFrench(x))
completeParadigmes=pd.pivot_table(lexiqueGold, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()

'''Identification des cases présentes initialement'''
completeParadigmes=completeParadigmes.loc[:,completeParadigmes.columns.isin(initialParadigmesColumns)]

'''Mise en liste des formes de références'''
goldTestForms=pd.melt(completeParadigmes[completeParadigmes["lexeme"].isin(listeLexemes)],id_vars=["lexeme"]).dropna()
goldTestForms["lexeme-case"]=goldTestForms["lexeme"]+"-"+goldTestForms["case"]
goldTestForms.drop(labels=["lexeme","case"],axis=1,inplace=True)
goldTestForms.set_index(["lexeme-case"],inplace=True)

'''Extraction des formes de références pertinentes'''
goldForms=goldTestForms.loc[~goldTestForms.index.isin(initialFormsIndex)]
goldFormsIndex=goldForms.index.tolist()

In [18]:
def countSplits(dfForms):
    dfForms.loc[:,"split"]=dfForms.loc[:,"value"].str.split(",")
    return dfForms["split"].str.len().sum()


In [19]:
'''Soustraire les formes initiales des prédictions'''
finalForms=predictedForms.loc[~predictedForms.index.isin(initialFormsIndex)]
finalFormsIndex=finalForms.index.tolist()

In [20]:
finalForms["value"]=finalForms["value"].apply(lambda x: checkFrench(x))

('pb avec un glide final', u'afilj')
('pb avec un glide final', u'apOstazj')
('pb avec un glide final', u'aparj')
('pb avec un glide final', u'armOrj')
('pb avec un glide final', u'asfiksj')
('pb avec un glide final', u'OtOpsj')
('pb avec un glide final', u'Sarj')
('pb avec un glide final', u'kOlOrj')
('pb avec un glide final', u'k\xf4fj')
('pb avec un glide final', u'k\xf4trarj')
('pb avec un glide final', u'kOpj')
('pb avec un glide final', u'disgrasj')
('pb avec un glide final', u'dEifj')
('pb avec un glide final', u'dEmarj')
('pb avec un glide final', u'EkskOrj')
('pb avec un glide final', u'fryktifj')
('pb avec un glide final', u'ZElifj')
('pb avec un glide final', u'istOrj')
('pb avec un glide final', u'likEfj')
('pb avec un glide final', u'mEfj')
('pb avec un glide final', u'Obvj')
('pb avec un glide final', u'Oktavj')
('pb avec un glide final', u'OrtOgrafj')
('pb avec un glide final', u'Osifj')
('pb avec un glide final', u'palj')
('pb avec un glide final', u'fOtOgrafj')
('pb av

('pb avec un glide final', u'Eputj')
('pb avec un glide final', u'Etydj')
('pb avec un glide final', u'afilj')
('pb avec un glide final', u'apOstazj')
('pb avec un glide final', u'aparj')
('pb avec un glide final', u'armOrj')
('pb avec un glide final', u'asfiksj')
('pb avec un glide final', u'OtOpsj')
('pb avec un glide final', u'Sarj')
('pb avec un glide final', u'kOlOrj')
('pb avec un glide final', u'k\xf4fj')
('pb avec un glide final', u'k\xf4trarj')
('pb avec un glide final', u'kOpj')
('pb avec un glide final', u'dEifj')
('pb avec un glide final', u'dEmarj')
('pb avec un glide final', u'EkskOrj')
('pb avec un glide final', u'fryktifj')
('pb avec un glide final', u'ZElifj')
('pb avec un glide final', u'istOrj')
('pb avec un glide final', u'likEfj')
('pb avec un glide final', u'mEfj')
('pb avec un glide final', u'Obvj')
('pb avec un glide final', u'Oktavj')
('pb avec un glide final', u'OrtOgrafj')
('pb avec un glide final', u'Osifj')
('pb avec un glide final', u'palj')
('pb avec un g

/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Est-ce que la surabondance est gérée entre l'échantillon et la référence ?

In [21]:
nbInitialForms=countSplits(initialForms)
nbGoldForms=countSplits(goldForms)
print nbGoldForms, "formes à prédire à partir de",nbInitialForms,"formes de",nbLexemes,"lexèmes"

198748 formes à prédire à partir de 29754 formes de 4574 lexèmes


/Users/gilles/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:284: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/gilles/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:461: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [22]:
'''Calculer les sur/sous-générations'''
underGeneration=goldForms.loc[~goldForms.index.isin(finalFormsIndex)]
overGeneration=finalForms.loc[~finalForms.index.isin(goldFormsIndex)]
print (countSplits(underGeneration),countSplits(overGeneration))

(44021, 3611)


/Users/gilles/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:545: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [23]:
'''Réduire les prédictions et la référence aux cases communes'''
compareForms=finalForms.loc[finalForms.index.isin(goldFormsIndex)].copy()
actualForms=goldForms.loc[goldForms.index.isin(finalFormsIndex)]
len(compareForms),len(actualForms)

(154536, 154536)

In [24]:
'''Créer un tableau pour les comparaisons'''
compareForms.loc[:,"right"]=actualForms.loc[:,"value"]
countSplits(compareForms)

154540

In [25]:
'''Séparer les cases identiques des cases différentes'''
sameForms=compareForms[compareForms["value"]==compareForms["right"]]
diffForms=compareForms[compareForms["value"]!=compareForms["right"]]

In [26]:
print (countSplits(sameForms))
print (countSplits(diffForms))

146742
7798


In [83]:
features.add_config('../bdlexique.ini')
fs=features.FeatureSystem('phonemes')

def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

class formesPatron:
    '''
    Accumulateur de formes correspondant à un patron pour calcul de la Généralisation Minimale (cf. MGL)
    '''
    def __init__(self):
        self.formes=[]

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterForme(self,forme):
        self.formes.append(forme)
        
    def calculerGM(self):
        minLongueur=len(min(self.formes, key=len))
        maxLongueur=len(max(self.formes, key=len))
#        if debug: print (minLongueur, maxLongueur, file=logfile)
        positions=[]
        if maxLongueur>minLongueur:
            positions.append("*")
        for i in xrange(minLongueur, 0, -1):
            phonemes=set([x[-i] for x in self.formes])
#            if debug: print (phonemes, file=logfile)
            if "." in phonemes:
                positions.append(".")
            else:
                print u"phonèmes",phonemes
                positions.append("".join(fs.lattice[phonemes].extent))
        return patron2regexp(positions)

class pairePatrons:
    '''
    Accumulateur de triplets (f1,f2,patron) correspondant à une paire pour calcul des Généralisations Minimales (cf. MGL)
    '''
    def __init__(self,case1,case2):
        self.patrons1={}
        self.patrons2={}
        self.case1=case1
        self.case2=case2

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterFormes(self,forme1,forme2,patron):
#        print (forme1,forme2,patron, file=logfile)
        patron12=patron
        (pat1,pat2)=patron.split("-")
        patron21=pat2+"-"+pat1
#        print (patron12,patron21, file=logfile)
        if not patron12 in self.patrons1:
            self.patrons1[patron12]=formesPatron()
        self.patrons1[patron12].ajouterForme(forme1)
        if not patron21 in self.patrons2:
            self.patrons2[patron21]=formesPatron()
        self.patrons2[patron21].ajouterForme(forme2)
        
        
    def calculerGM(self):
        resultat1={}
        for patron in self.patrons1:
#            if debug: print ("patron1", patron, file=logfile)
            resultat1[patron]=self.patrons1[patron].calculerGM()
        resultat2={}
        for patron in self.patrons2:
#            if debug: print ("patron2", patron, file=logfile)
            resultat2[patron]=self.patrons2[patron].calculerGM()
        return (resultat1,resultat2) 

class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
#        else:
#            if debug: print ("le numéro de forme n'est pas dans [1,2]",n, file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
        self.classeCF={}
        self.nbClasseCF={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeFormeCF=[]
        regleFormeCF=""
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            filterF1=".*"+patron.split("-")[0]+"$"
            if re.match(filterF1,forme1):
                classeFormeCF.append(patron)
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleFormeCF=patron
            filterF1=self.patrons[patron]
            if re.match(filterF1,forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseFormeCF=", ".join(classeFormeCF)
        if not idClasseFormeCF in self.classeCF:
            self.classeCF[idClasseFormeCF]={}
            self.nbClasseCF[idClasseFormeCF]=0
        if not regleFormeCF in self.classeCF[idClasseFormeCF]:
            self.classeCF[idClasseFormeCF][regleFormeCF]=0
        self.nbClasseCF[idClasseFormeCF]+=1
        self.classeCF[idClasseFormeCF][regleFormeCF]+=1
        
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme,contextFree=False):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if contextFree:
                filterF1=".*"+patron.split("-")[0]+"$"
            else:
                filterF1=self.patrons[patron]
            if re.match(filterF1,forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if contextFree:
                nbClasse=self.nbClasseCF
                classe=self.classeCF
            else:
                nbClasse=self.nbClasse
                classe=self.classe
            if idClasseForme in nbClasse:
                nTotal=nbClasse[idClasseForme]
                for patron in classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(classe[idClasseForme][patron])/nTotal
            else:
#                if debug: 
#                    print (forme, file=logfile)
#                    print ("pas de classe",idClasseForme, file=logfile)
#                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
#        else:
#            if debug:
#                print (forme, file=logfile) 
#                print ("pas de patron", file=logfile)
        return sortieForme
        

def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)



def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


In [79]:
validPhonemes=list(fs.supremum.concept.extent)
validPhonemes

[u'p',
 u't',
 u'k',
 u'b',
 u'd',
 u'g',
 u'f',
 u's',
 u'S',
 u'v',
 u'z',
 u'Z',
 u'm',
 u'n',
 u'J',
 u'j',
 u'l',
 u'r',
 u'w',
 u'H',
 u'i',
 u'y',
 u'E',
 u'e',
 u'9',
 u'2',
 u'6',
 u'a',
 u'u',
 u'O',
 u'o',
 u'\xea',
 u'\xfb',
 u'\xe2',
 u'\xf4']

In [75]:
temp=diffForms.reset_index()

In [88]:
temp["lexeme"]=temp["lexeme-case"].str.rsplit("-",n=1).str[0]
temp["case"]=temp["lexeme-case"].str.rsplit("-",n=1).str[1]

casesDiff=temp["case"].unique().tolist()
resultTemp={}
for case in casesDiff:
    print "case",case
    paradigmePaire=temp[temp["case"]==case][["value","right","lexeme"]]
    paradigmePaire.rename(columns={"value":case},inplace=True)
    
    paradigmePaire=splitCellMates(splitCellMates(paradigmePaire,case),"right")
    resultTemp[case]=rapports(paradigmePaire)

case ai1P
           ai1P       right       lexeme
0     akursisam     akursim    accourcir
1      blEtisam      blEtim      blettir
2       kajEtam     kaj6tam    cailleter
3    divErtisam    divErtim     divertir
4       dEparam    dEpartim     départir
5       mêt6nam       mêtêm    maintenir
6       mEdizam       mEdim       médire
7      pErmEtam      pErmim    permettre
8    r6kôdHizam  r6kôdHizim   reconduire
9       r6kuwam     r6kuzim     recoudre
10   r6garnisam    r6garnim     regarnir
11  rEsplâdisam   rEsplâdim   resplendir
12    r6vOmisam     r6vOmim      revomir
13      r6vulam     r6vulym    revouloir
14     rEâboSam    rEâbOSam  réembaucher
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'\xe2'])
phonèmes set([u'b'])
phonèmes set([u'o'])
phonèmes set([u'S'])
phonèmes set([u'a'])
phonèmes set([u'm'])
phonèmes set([u'r'])
phonèmes set([u'6'])
phonèmes set([u'k'])
phonèmes set([u'u'])
phonèmes set([u'w'])
phonèmes set([u'a'])
phonèmes set([u'm'])
phonèmes set([u'

phonèmes set([u'w'])
phonèmes set([u'E'])
phonèmes set([u'r'])
phonèmes set([u'a'])
phonèmes set([u'b'])
phonèmes set([u'\xf4'])
phonèmes set([u'd'])
phonèmes set([u'E'])
phonèmes set([u'r'])
phonèmes set([u'a', u'E', u'f'])
phonèmes set([u's', u'b', u'r'])
phonèmes set([u'a', u'E', u'O'])
phonèmes set([u'm', u't', u'n'])
phonèmes set([u'i'])
phonèmes set([u'r'])
phonèmes set([u'd'])
phonèmes set([u'E'])
phonèmes set([u'p'])
phonèmes set([u'a'])
phonèmes set([u'r'])
phonèmes set([u't'])
phonèmes set([u'i'])
phonèmes set([u'r'])
phonèmes set([u'p'])
phonèmes set([u'a'])
phonèmes set([u'r'])
phonèmes set([u'k'])
phonèmes set([u'u'])
phonèmes set([u'r'])
phonèmes set([u'y'])
phonèmes set([u'r'])
phonèmes set([u'p'])
phonèmes set([u'E'])
phonèmes set([u'r'])
phonèmes set([u'm'])
phonèmes set([u'i'])
phonèmes set([u'r'])
phonèmes set([u'p', u'r'])
phonèmes set([u'a', u'\xe2'])
phonèmes set([u'p', u't'])
phonèmes set([u'r', u'l'])
phonèmes set([u'i'])
phonèmes set([u'j'])
phonèmes set([u'E']

phonèmes set([u'6'])
phonèmes set([u'v'])
phonèmes set([u'u'])
phonèmes set([u'd'])
phonèmes set([u'r'])
phonèmes set([u'\xf4'])
phonèmes set([u'r', u'd'])
phonèmes set([u'a', u'E'])
phonèmes set([u'g'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'r'])
phonèmes set([u'\xf4'])
phonèmes set([u'H', u'r', u'l', u'w', u'p'])
phonèmes set([u'i', u'a', u'E'])
phonèmes set([u'r'])
phonèmes set([u'\xf4'])
phonèmes set([u'p'])
phonèmes set([u'E'])
phonèmes set([u'r'])
phonèmes set([u'm'])
phonèmes set([u'E'])
phonèmes set([u't'])
phonèmes set([u'r'])
phonèmes set([u'\xf4'])
phonèmes set([u'd'])
phonèmes set([u'E'])
phonèmes set([u'p'])
phonèmes set([u'a'])
phonèmes set([u'r'])
phonèmes set([u't'])
phonèmes set([u'i'])
phonèmes set([u'r'])
phonèmes set([u'\xf4'])
phonèmes set([u'\xe2'])
phonèmes set([u'n'])
phonèmes set([u'i'])
phonèmes set([u'r'])
phonèmes set([u'\xf4'])
phonèmes set([u'\xe2'])
phonèmes set([u'v'])
phonèmes set([u'E'])
phonèmes set([u'r'])
phonèmes set([u'\xf4'])
c

            fi2P        right        lexeme
0         abôdre      abôd6re       abonder
1     akursis6re     akursire     accourcir
2        aJ6l6re      aJEl6re       agneler
3          vatre          ire         aller
4    apOstazj6re   apOstazire    apostasier
5       apHij6re       apHire       appuyer
6      armOrj6re     armOrire      armorier
7     asfiksj6re    asfiksire     asphyxier
8    atErmwaj6re    atErmware     atermoyer
9      OtOpsj6re     OtOpsire     autopsier
10      bâk6t6re     bâkEt6re     banqueter
11     klam6s6re    klamEs6re      clamecer
12     kOlOrj6re     kOlOrire      colorier
13      kOk6t6re     kOkEt6re      coqueter
14   divErtis6re    divErtire      divertir
16    dESik6t6re   dESikEt6re   déchiqueter
18       dEgrere      dEgrEre       dégréer
19    dEpak6t6re   dEpakEt6re    dépaqueter
20      dEpar6re    dEpartire      départir
22    dEzâp6z6re   dEzâpEz6re    désempeser
23       âp6z6re      âpEz6re       empeser
24       ânal6re        ânire   

phonèmes set([u'd'])
phonèmes set([u'r'])
phonèmes set([u'a'])
phonèmes set([u'p', u'r', u'm', u'l'])
phonèmes set([u'E', u'O', u'6'])
phonèmes set([u'j', u'l', u'n'])
phonèmes set([u'E', u'6'])
phonèmes set([u'r'])
phonèmes set([u'a'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'\xe2'])
phonèmes set([u'b'])
phonèmes set([u'O'])
phonèmes set([u'S'])
phonèmes set([u'6'])
phonèmes set([u'r'])
phonèmes set([u'a'])
phonèmes set([u'\xe2'])
phonèmes set([u'n'])
phonèmes set([u'i'])
phonèmes set([u'r'])
phonèmes set([u'a'])
phonèmes set([u'p'])
phonèmes set([u'E'])
phonèmes set([u'r'])
phonèmes set([u'm'])
phonèmes set([u'E'])
phonèmes set([u't'])
phonèmes set([u'r'])
phonèmes set([u'a'])
phonèmes set([u'H', u'r', u'l', u'w', u'p'])
phonèmes set([u'a', u'i', u'E'])
phonèmes set([u'r'])
phonèmes set([u'a'])
phonèmes set([u'p'])
phonèmes set([u'E'])
phonèmes set([u'r'])
phonèmes set([u's'])
phonèmes set([u'6'])
phonèmes set([u'v'])
phonèmes set([u'r'])
phonèmes set([u'a'])
phonème

...

phonèmes set([u'i'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'i'])
phonèmes set([u'r'])
phonèmes set([u'E'])
case pc3S
            pc3S        right       lexeme
0         abôdrE      abôd6rE      abonder
1     akursis6rE     akursirE    accourcir
2        aJ6l6rE      aJEl6rE      agneler
3          vatrE          irE        aller
4    apOstazj6rE   apOstazirE   apostasier
5       aparj6rE      aparirE     apparier
6       apHij6rE       apHirE      appuyer
7      armOrj6rE     armOrirE     armorier
8     asfiksj6rE    asfiksirE    asphyxier
10   atErmwaj6rE    atErmwarE    atermoyer
11     OtOpsj6rE     OtOpsirE    autopsier
12     blEtis6rE      blEtirE      blettir
13       Sarj6rE       SarirE     charrier
14     kOlOrj6rE     kOlOrirE     colorier
15    kôtrarj6rE    kôtrarirE   contrarier
16   divErtis6rE    divErtirE     divertir
19       dEgrerE      dEgrErE      dégréer
20     dEmarj6rE     dEmarirE     démarier
21      dEpar6rE    dEpartirE     départir
24  

phonèmes set([u'a', u'E', u'g'])
phonèmes set([u's', u'b', u'l'])
phonèmes set([u'a', u'\xe2', u'E', u'O'])
phonèmes set([u'p', u't', u'n'])
phonèmes set([u'j'])
phonèmes set([u'\xf4'])
phonèmes set([u's'])
phonèmes set([u'u'])
phonèmes set([u's'])
phonèmes set([u'k'])
phonèmes set([u'r'])
phonèmes set([u'i'])
phonèmes set([u'\xf4'])
phonèmes set([u'a', u'E', u'g'])
phonèmes set([u's', u'b', u'l'])
phonèmes set([u'a', u'\xe2', u'E', u'O'])
phonèmes set([u'p', u't', u'n'])
phonèmes set([u'i'])
phonèmes set([u's'])
phonèmes set([u'\xf4'])
phonèmes set([u'r', u'd'])
phonèmes set([u'a', u'E'])
phonèmes set([u'g'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'\xf4'])
phonèmes set([u'k', u'r', u'Z', u'l'])
phonèmes set([u'a', u'i', u'6'])
phonèmes set([u'k', u'j', u'Z', u's', u'v'])
phonèmes set([u'6'])
phonèmes set([u'p', u't', u'l'])
phonèmes set([u'\xf4'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u't'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'

phonèmes set([u'r'])
phonèmes set([u'\xe2'])
phonèmes set([u'p'])
phonèmes set([u'l'])
phonèmes set([u'i'])
phonèmes set([u's'])
phonèmes set([u'e'])
phonèmes set([u'r'])
phonèmes set([u'6'])
phonèmes set([u'd'])
phonèmes set([u'i'])
phonèmes set([u't'])
phonèmes set([u'a'])
phonèmes set([u'l'])
phonèmes set([u'e'])
phonèmes set([u'k', u'r', u'Z', u'l'])
phonèmes set([u'a', u'i', u'6'])
phonèmes set([u'k', u'j', u'Z', u's', u'v'])
phonèmes set([u'6'])
phonèmes set([u'p', u't', u'l'])
phonèmes set([u'e'])
phonèmes set([u'r'])
phonèmes set([u'\xe2'])
phonèmes set([u'p'])
phonèmes set([u'l'])
phonèmes set([u'i'])
phonèmes set([u'j'])
phonèmes set([u'e'])
phonèmes set([u'p'])
phonèmes set([u'E'])
phonèmes set([u'r'])
phonèmes set([u's'])
phonèmes set([u'6'])
phonèmes set([u'v'])
phonèmes set([u'e'])
phonèmes set([u'a', u'E', u'g', u'f'])
phonèmes set([u's', u'b', u'r', u'l'])
phonèmes set([u'a', u'\xe2', u'E', u'O'])
phonèmes set([u'p', u'm', u't', u'n'])
phonèmes set([u'i'])
phonèmes set(

phonèmes set([u'b'])
phonèmes set([u'\xf4'])
phonèmes set([u'd'])
phonèmes set([u'y'])
phonèmes set([u'p'])
phonèmes set([u'a'])
phonèmes set([u'r'])
phonèmes set([u'k'])
phonèmes set([u'u'])
phonèmes set([u'r'])
phonèmes set([u'i'])
phonèmes set([u'a', u'E', u'g', u'f'])
phonèmes set([u's', u'b', u'r', u'l'])
phonèmes set([u'a', u'\xe2', u'E', u'O'])
phonèmes set([u'p', u'm', u't', u'n'])
phonèmes set([u'j'])
phonèmes set([u'e'])
phonèmes set([u'k', u'b', u'g', u'v'])
phonèmes set([u'a', u'E', u'u', u'l'])
phonèmes set([u'r', u'E'])
phonèmes set([u's', u't', u'n'])
phonèmes set([u'i'])
phonèmes set([u's'])
phonèmes set([u'e'])
phonèmes set([u'd'])
phonèmes set([u'E'])
phonèmes set([u'p'])
phonèmes set([u'a'])
phonèmes set([u'r'])
phonèmes set([u'e'])
phonèmes set([u'f', u't', u'v'])
phonèmes set([u'i', u'u', u'6'])
phonèmes set([u'S', u'l', u'n'])
phonèmes set([u'e'])
phonèmes set([u'v'])
phonèmes set([u'a'])
phonèmes set([u't'])
phonèmes set([u'y'])
phonèmes set([u'r', u'l'])
phonème

phonèmes set([u'n'])
phonèmes set([u'a'])
phonèmes set([u'k'])
phonèmes set([u'i'])
phonèmes set([u's'])
phonèmes set([u'a'])
phonèmes set([u'f'])
phonèmes set([u'l'])
phonèmes set([u'6'])
phonèmes set([u'r'])
phonèmes set([u'z'])
phonèmes set([u'j'])
phonèmes set([u'6'])
phonèmes set([u't'])
phonèmes set([u'H', u'r', u'l', u'w', u'p'])
phonèmes set([u'i', u'a', u'E'])
phonèmes set([u'j'])
phonèmes set([u'\xe2'])
phonèmes set([u'n'])
phonèmes set([u'a'])
phonèmes set([u'l'])
phonèmes set([u's'])
phonèmes set([u'u'])
phonèmes set([u's'])
phonèmes set([u'k'])
phonèmes set([u'r'])
phonèmes set([u'i'])
phonèmes set([u'p'])
phonèmes set([u'a'])
phonèmes set([u'r'])
phonèmes set([u'k'])
phonèmes set([u'u'])
phonèmes set([u'r'])
phonèmes set([u'i'])
phonèmes set([u's'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u't'])
phonèmes set([u'r'])
phonèmes set([u'\xea'])
phonèmes set([u'k'])
phonèmes set([u'l'])
phonèmes set([u'O'])
phonèmes set([u's'])
phonèmes set([u't'])
phonèmes set(

In [104]:
resultTemp["pi1S"][0].nbClasse

{u'-, j-': 14,
 u'-j': 2,
 u'-j, wa-Hi': 1,
 u'6..-2..': 1,
 u'6..-9..': 4,
 u'O.-o.': 22,
 u'O...-o...': 1,
 u'a-E': 1,
 u'aki-E': 1,
 u'i-': 1,
 u'j-': 4,
 u'ji-': 1,
 u'nal-vE': 1,
 u'o.-O.': 9,
 u's-': 6,
 u't-': 1,
 u'ul-2': 1,
 u'z-': 3}

In [102]:
paradigmePaire

,ps3S,right,lexeme
0,afl6r,afl9r,affleurer
1,anot,anOt,annoter
2,apHij,apHi,appuyer
3,atErmwaj,atErmwa,atermoyer
4,bEzot,bEzOt,baisoter
5,bOZ,boZ,bauger
6,bwajOt,bwajot,boyauter
7,brHi,brHis,bruire
8,kalf6tr,kalf2tr,calfeutrer
9,sirkôsis,sirkôsiz,circoncire


In [27]:
'''Sauvegarder les comparatifs'''
overGeneration.to_csv(path_or_buf=analysisPrefix+"-overGeneration%s.csv"%etapeSwim,encoding="utf8")
underGeneration.to_csv(path_or_buf=analysisPrefix+"-underGeneration%s.csv"%etapeSwim,encoding="utf8")
sameForms.to_csv(path_or_buf=analysisPrefix+"-sameForms%s.csv"%etapeSwim,encoding="utf8")
diffForms.to_csv(path_or_buf=analysisPrefix+"-diffForms%s.csv"%etapeSwim,encoding="utf8")
 

In [28]:
'''Transformer les surabondances en liste'''
diffForms.loc[:,"split-value"]=diffForms.loc[:,"value"].str.split(",")
diffForms.loc[:,"split-right"]=diffForms.loc[:,"right"].str.split(",")

'''Transformer les surabondances en set()'''
diffForms.loc[:,"split-value"]=diffForms.loc[:,"split-value"].apply(set)
diffForms.loc[:,"split-right"]=diffForms.loc[:,"split-right"].apply(set)
    
'''Calculer le nombre de formes (y compris surabondances)'''
nbValues=diffForms["split-value"].str.len().sum()
nbRights=diffForms["split-right"].str.len().sum()

'''Calculer les identités et les inclusions'''
nbIdenticalSets=diffForms[diffForms["split-value"]==diffForms["split-right"]]["split-value"].str.len().sum()
nbIncludedSets=diffForms[diffForms["split-value"]<diffForms["split-right"]]["split-value"].str.len().sum()
nbWrongForms=(nbValues-nbIdenticalSets-nbIncludedSets)
underBonus=(nbRights-nbIdenticalSets-nbIncludedSets)

In [29]:
diffForms[diffForms.index.str.contains("-ii1P")]

,value,right,split,split-value,split-right
lexeme-case,,,,,
aller-ii1P,vatjô,aljô,[vatjô],{vatjô},{aljô}
cailleter-ii1P,kajEtjô,kaj6tjô,[kajEtjô],{kajEtjô},{kaj6tjô}
cliqueter-ii1P,klikEtjô,klik6tjô,[klikEtjô],{klikEtjô},{klik6tjô}
dégréer-ii1P,dEgrejô,dEgrEjô,[dEgrejô],{dEgrejô},{dEgrEjô}
départir-ii1P,dEparjô,dEpartjô,[dEparjô],{dEparjô},{dEpartjô}
flétrir-ii1P,flEtrijô,flEtrisjô,[flEtrijô],{flEtrijô},{flEtrisjô}
griveler-ii1P,grivEljô,griv6ljô,[grivEljô],{grivEljô},{griv6ljô}
javeler-ii1P,ZavEljô,Zav6ljô,[ZavEljô],{ZavEljô},{Zav6ljô}
percevoir-ii1P,pErswajô,pErs6vjô,[pErswajô],{pErswajô},{pErs6vjô}


In [30]:
UG=countSplits(underGeneration)+underBonus
OG=countSplits(overGeneration)
TP=countSplits(sameForms)+nbIdenticalSets+nbIncludedSets
FP=nbWrongForms
resultCharacteristics=(UG,OG,TP,FP)
recall=float(TP)/(UG+TP)
precision=float(TP)/(OG+TP+FP)
fMeasure=2*recall*precision/(recall+precision)
resultMeasures=(precision,recall,fMeasure)
print ("UG",UG ,"OG",OG,"TP",TP,"FP",FP)
print UG+TP
print ("recall", recall, "precision", precision)
print (fMeasure)

('UG', 51812, 'OG', 3611, 'TP', 146936, 'FP', 7604)
198748
('recall', 0.7393080685088655, 'precision', 0.9290867588570417)
0.823403820128


In [ ]:
nomFichierResultats=filePrefix+"Longitudinal-X-Resultats-Recalcul.yaml"
if os.path.isfile(nomFichierResultats):
    with open(nomFichierResultats, 'r') as stream:
        resultats=yaml.load(stream)
        if not resultats:
            resultats={}
else:
    resultats={}
    
if casesType:
    sampleExt=casesType
else:
    sampleExt=sampleType
sampleId=samplePrefix.strip("Longitudinal").strip("-")+sampleExt
print resultats

In [ ]:
if not sampleId in resultats:
    resultats[sampleId]={}
    
etape=etapeSwim.strip("-")
resultats[sampleId][etape]={}
resultats[sampleId][etape]["nbGoldForms"]=nbGoldForms
resultats[sampleId][etape]["nbInitialForms"]=nbInitialForms
resultats[sampleId][etape]["nbLexemes"]=nbLexemes
resultats[sampleId][etape]["UG"]=UG
resultats[sampleId][etape]["OG"]=OG
resultats[sampleId][etape]["TP"]=TP
resultats[sampleId][etape]["FP"]=FP
resultats[sampleId][etape]["Precision"]=precision
resultats[sampleId][etape]["Recall"]=recall
resultats[sampleId][etape]["F-Measure"]=fMeasure

yaml.safe_dump(resultats, file(nomFichierResultats, 'w'), encoding='utf-8', allow_unicode=True)

In [ ]:
fin=datetime.datetime.now()
print debut
print fin